In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


I'm reading in the results of the combine customer billing script which is a combination of all the files in the customer billing folder

In [4]:
df = pd.read_csv('customer_billing.csv')

In [5]:
df.head(10)

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
0,2019-01-01,PARENT001,CUST955,SERV010,PROD058,Product 058,6,$243.75
1,2019-01-01,PARENT100,CUST957,SERV014,PROD058,Product 058,7,$160.00
2,2019-01-01,PARENT303,CUST958,SERV001,PROD058,Product 058,4,$208.75
3,2019-01-01,PARENT208,CUST959,SERV014,PROD058,Product 058,19,$226.25
4,2019-01-01,PARENT208,CUST962,SERV004,PROD058,Product 058,16,$226.25
5,2019-01-01,PARENT068,CUST963,SERV014,PROD058,Product 058,5,$143.75
6,2019-01-01,PARENT386,CUST964,SERV007,PROD058,Product 058,3,$183.75
7,2019-01-01,PARENT297,CUST965,SERV015,PROD058,Product 058,13,$115.00
8,2019-01-01,PARENT324,CUST966,SERV020,PROD058,Product 058,11,$211.25
9,2019-01-01,PARENT335,CUST967,SERV005,PROD058,Product 058,12,$100.00


In [6]:
df.describe(include='all')

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
count,2782955,2782955,2782955,2782955,2782955,2782955,2.782955e+06,2782955
unique,121,500,1000,20,50,50,NaN,1242
top,2020-01-01,PARENT472,CUST256,SERV003,PROD059,Product 059,NaN,$189.00
freq,23308,10183,3143,141962,60829,60829,NaN,7642
mean,NaN,NaN,NaN,NaN,NaN,NaN,9.997117e+00,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,5.481457e+00,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+00,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+01,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.500000e+01,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782955 entries, 0 to 2782954
Data columns (total 8 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Billing Date         object
 1   Parent ID            object
 2   Customer ID          object
 3   Service ID           object
 4   Product ID           object
 5   Product Description  object
 6   Volume               int64 
 7   Charge               object
dtypes: int64(1), object(7)
memory usage: 169.9+ MB


This is an example of the type of dataset I'm constantly working with. It's a time-series dataset that consists of customer billing. One of the main tasks I have is to forecast customer volume and revenue.

In [8]:
# Convert 'Billing Date' to datetime
df['Billing Date'] = pd.to_datetime(df['Billing Date'])

In [9]:
# Convert 'Charge' to float
df['Charge'] = df['Charge'].str.replace('$', '').astype(float)

In [10]:
# Check to make sure the changes were made
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782955 entries, 0 to 2782954
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Billing Date         datetime64[ns]
 1   Parent ID            object        
 2   Customer ID          object        
 3   Service ID           object        
 4   Product ID           object        
 5   Product Description  object        
 6   Volume               int64         
 7   Charge               float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 169.9+ MB


In [11]:
df.head()

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
0,2019-01-01,PARENT001,CUST955,SERV010,PROD058,Product 058,6,243.75
1,2019-01-01,PARENT100,CUST957,SERV014,PROD058,Product 058,7,160.00
2,2019-01-01,PARENT303,CUST958,SERV001,PROD058,Product 058,4,208.75
3,2019-01-01,PARENT208,CUST959,SERV014,PROD058,Product 058,19,226.25
4,2019-01-01,PARENT208,CUST962,SERV004,PROD058,Product 058,16,226.25


We want to predict at the parent level because that's how we like to view our customer base

In [12]:
# Aggregate the data to the Parent ID level
df_grouped = df.groupby(['Billing Date', 'Parent ID', 'Service ID', 'Product ID']).agg({'Volume': 'sum', 'Charge': 'sum'}).reset_index()

In [13]:
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge
0,2014-01-01,PARENT001,SERV001,PROD002,17,82.0
1,2014-01-01,PARENT001,SERV001,PROD005,2,54.0
2,2014-01-01,PARENT001,SERV001,PROD007,6,56.0
3,2014-01-01,PARENT001,SERV001,PROD008,10,163.0
4,2014-01-01,PARENT001,SERV001,PROD011,19,163.0


In [14]:
df_grouped.shape

(2718874, 6)

In [15]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns to normalize
columns_to_normalize = ['Volume', 'Charge']

# Normalize the selected columns
df_grouped[columns_to_normalize] = scaler.fit_transform(df_grouped[columns_to_normalize])

In [16]:
# Make sure data is normalized
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382


In [17]:
# Ensure df_grouped is sorted by 'Billing Date' and Parent ID
df_grouped = df_grouped.sort_values(by=['Billing Date', 'Parent ID'])

# Feature Engineering

To perform feature engineering on the `df_grouped` dataframe, I wanted to consider the following:

1. Time-based Features:
    - Extract additional time-based features from the 'Billing Date' column, such as day of the week, day of the month, or hour of the day.

2. Rolling Window Statistics:
    - Compute rolling window statistics, such as rolling mean, rolling sum, or rolling standard deviation, for the 'Volume' and 'Charge' columns. This can provide insights into the trend and variability of the data over time.

3. Lagged Features:
    - Create lagged features by shifting the 'Volume' and 'Charge' columns by a certain number of time periods. This can capture the historical values of these variables and their impact on future values.

4. Categorical Encoding:
    - If there are categorical columns in the dataframe, such as 'Parent ID', 'Service ID', or 'Product ID', you can encode them using techniques like one-hot encoding or label encoding. This can enable the use of categorical variables in machine learning models.

5. Seasonality Features:
    - Identify and incorporate seasonality patterns in the data. For example, you can create binary variables indicating whether a particular month or quarter is associated with a specific season or holiday.

In [18]:
# Create columns for Year, Month, and Quarter of Billing Date
df_grouped['Year'] = df_grouped['Billing Date'].dt.year
df_grouped['Month'] = df_grouped['Billing Date'].dt.month
df_grouped['Quarter'] = df_grouped['Billing Date'].dt.quarter

In [19]:
# Make sure the columns were added
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1


In [29]:
# Create rolling averages for 'Volume' and 'Charge' for the past 3 months
df_grouped['Volume Rolling 3 Month Avg'] = df_grouped.groupby('Parent ID')['Volume'].transform(lambda x: x.rolling(3).mean())
df_grouped['Charge Rolling 3 Month Avg'] = df_grouped.groupby('Parent ID')['Charge'].transform(lambda x: x.rolling(3).mean())

In [31]:
# Calculate the difference between 'Volume' and 'Volume Rolling 3 Month Avg'
df_grouped['Volume Rolling 3 Month Avg Diff'] = df_grouped['Volume'] - df_grouped['Volume Rolling 3 Month Avg']

# Calculate the difference between 'Charge' and 'Charge Rolling 3 Month Avg'
df_grouped['Charge Rolling 3 Month Avg Diff'] = df_grouped['Charge'] - df_grouped['Charge Rolling 3 Month Avg']

In [32]:
# Make sure the calculations were made
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1,NaN,NaN,NaN,NaN
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1,NaN,NaN,NaN,NaN
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1,0.126437,0.019499,-0.040230,-0.011142
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1,0.086207,0.057103,0.068966,0.100279
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1,0.183908,0.107707,0.126437,0.049675


In [37]:
# Create a column to calculate the percentage change in 'Volume' from the previous month
df_grouped['Volume % Change'] = df_grouped.groupby('Parent ID')['Volume'].pct_change()

# Create a column to calculate the percentage change in 'Charge' from the previous month
df_grouped['Charge % Change'] = df_grouped.groupby('Parent ID')['Charge'].pct_change()

# Create a column to calculate the percentage change in 'Volume' from the previous quarter
df_grouped['Volume % Change Qtr'] = df_grouped.groupby('Parent ID')['Volume'].pct_change(periods=3)

# Create a column to calculate the percentage change in 'Charge' from the previous quarter
df_grouped['Charge % Change Qtr'] = df_grouped.groupby('Parent ID')['Charge'].pct_change(periods=3)

# Create a column to calculate the percentage change in 'Volume' from the previous year
df_grouped['Volume % Change Yr'] = df_grouped.groupby('Parent ID')['Volume'].pct_change(periods=12)

# Create a column to calculate the percentage change in 'Charge' from the previous year
df_grouped['Charge % Change Yr'] = df_grouped.groupby('Parent ID')['Charge'].pct_change(periods=12)

In [38]:
# Check out the new columns
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Holiday Season,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1,NaN,NaN,NaN,NaN,0,-0.9375,-0.875000,NaN,NaN,NaN,NaN
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1,0.126437,0.019499,-0.040230,-0.011142,0,4.0000,0.500000,NaN,NaN,NaN,NaN
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1,0.086207,0.057103,0.068966,0.100279,0,0.8000,17.833333,-0.4375,2.53125,NaN,NaN
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1,0.183908,0.107707,0.126437,0.049675,0,1.0000,0.000000,17.0000,27.25000,NaN,NaN


In [35]:
# Make a column for any months that fall in the holiday season
# November and December are considered the holiday season
df_grouped['Holiday Season'] = df_grouped['Month'].apply(lambda x: 1 if x in [11, 12] else 0)

In [36]:
# Show a sample of holiday season months
df_grouped[df_grouped['Holiday Season'] == 1].head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Holiday Season
224788,2014-11-01,PARENT001,SERV002,PROD053,0.310345,0.147632,2014,11,4,0.224138,0.146240,0.086207,0.001393,1
224789,2014-11-01,PARENT001,SERV005,PROD052,0.310345,0.034819,2014,11,4,0.281609,0.096100,0.028736,-0.061281,1
224790,2014-11-01,PARENT001,SERV007,PROD053,0.275862,0.115599,2014,11,4,0.298851,0.099350,-0.022989,0.016249,1
224791,2014-11-01,PARENT001,SERV007,PROD058,0.034483,0.164345,2014,11,4,0.206897,0.104921,-0.172414,0.059424,1
224792,2014-11-01,PARENT001,SERV009,PROD002,0.189655,0.041783,2014,11,4,0.166667,0.107242,0.022989,-0.065460,1


In [39]:
# Mark any months that fall into the covid-19 pandemic
# The pandemic started in March 2020
# We'll end the pandemic in December 2022
df_grouped['Covid-19 Pandemic'] = df_grouped['Billing Date'].apply(lambda x: 1 if x >= pd.Timestamp('2020-03-01') and x <= pd.Timestamp('2022-12-31') else 0)


In [42]:
# Show a sample of months during the pandemic
df_grouped[df_grouped['Covid-19 Pandemic'] == 1].sample(5)

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,...,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Holiday Season,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr,Covid-19 Pandemic
1915303,2021-02-01,PARENT104,SERV006,PROD046,0.327586,0.486908,2021,2,1,0.252874,...,0.074713,0.248816,0,0.266667,2.364774,-0.173913,-0.133796,-0.344828,-0.103820,1
2057489,2021-08-01,PARENT277,SERV010,PROD048,0.258621,0.223677,2021,8,3,0.103448,...,0.155172,0.046379,0,6.500000,0.889412,-0.166667,-0.008336,2.750000,6.950495,1
2339998,2022-09-01,PARENT069,SERV002,PROD045,0.258621,0.047354,2022,9,3,0.206897,...,0.051724,-0.107892,0,2.000000,-0.820486,0.071429,-0.603730,0.875000,-0.745127,1
2110377,2021-10-01,PARENT459,SERV015,PROD049,0.000000,0.208635,2021,10,4,0.114943,...,-0.114943,0.015669,0,-1.000000,0.520812,-1.000000,-0.139575,-1.000000,3.115385,1
1966373,2021-04-01,PARENT254,SERV020,PROD006,0.034483,0.163510,2021,4,2,0.189655,...,-0.155172,-0.019429,0,-0.888889,0.279564,-0.882353,-0.022481,-0.600000,-0.400102,1


In [48]:
# Create a column to calculate the number of months since the start of the pandemic
# Only consider months after the start of the pandemic  (March 2020)
df_grouped['Months Since Pandemic'] = (df_grouped['Billing Date'] - pd.Timestamp('2020-03-01')).dt.days / 30.44


In [49]:
# Show a sample of months since the start of the pandemic
df_grouped[df_grouped['Months Since Pandemic'] > 0].sample(5)

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,...,Charge Rolling 3 Month Avg Diff,Holiday Season,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr,Covid-19 Pandemic,Months Since Pandemic
2387620,2022-11-01,PARENT143,SERV017,PROD007,0.051724,0.209192,2022,11,4,0.074713,...,0.046147,1,0.500000,-0.143672,-0.700000,-0.018301,0.000000,-0.266602,1,32.030223
1874158,2020-12-01,PARENT201,SERV016,PROD006,0.241379,0.183844,2020,12,4,0.195402,...,0.066992,1,1.800000,0.573302,1.800000,0.074044,0.400000,0.344196,1,9.034166
2529942,2023-05-01,PARENT295,SERV013,PROD047,0.293103,0.065669,2023,5,2,0.206897,...,-0.097609,0,-0.055556,-0.652286,0.133333,0.511218,0.888889,-0.691830,0,37.976347
1883808,2020-12-01,PARENT419,SERV015,PROD038,0.137931,0.029944,2020,12,4,0.103448,...,-0.093547,1,0.000000,-0.739709,1.000000,-0.868260,-0.200000,-0.884780,1,9.034166
2284688,2022-06-01,PARENT338,SERV011,PROD037,0.258621,0.029805,2022,6,2,0.206897,...,-0.067595,0,0.875000,-0.713137,0.000000,-0.439791,1.500000,-0.901564,1,27.003942


In [54]:
# Create dummy variables for 'Parent ID', 'Service ID', 'Product ID', 'Year', 'Month', and 'Quarter'
df_grouped = pd.get_dummies(df_grouped, columns=['Parent ID', 'Service ID', 'Product ID', 'Year', 'Month', 'Quarter'], drop_first=True)

In [55]:
# Make sure the dummy variables were created
df_grouped.head()

,Billing Date,Volume,Charge,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Holiday Season,Volume % Change,Charge % Change,...,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Quarter_2,Quarter_3,Quarter_4
0,2014-01-01,0.275862,0.044568,NaN,NaN,NaN,NaN,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2014-01-01,0.017241,0.005571,NaN,NaN,NaN,NaN,0,-0.9375,-0.875000,...,False,False,False,False,False,False,False,False,False,False
2,2014-01-01,0.086207,0.008357,0.126437,0.019499,-0.040230,-0.011142,0,4.0000,0.500000,...,False,False,False,False,False,False,False,False,False,False
3,2014-01-01,0.155172,0.157382,0.086207,0.057103,0.068966,0.100279,0,0.8000,17.833333,...,False,False,False,False,False,False,False,False,False,False
4,2014-01-01,0.310345,0.157382,0.183908,0.107707,0.126437,0.049675,0,1.0000,0.000000,...,False,False,False,False,False,False,False,False,False,False


In [56]:
df_grouped.shape

(2718874, 607)

In [57]:
# Save the cleaned data to a new CSV file
df_grouped.to_csv('customer_billing_cleaned.csv', index=False)